**Table of contents**

* [Parts of Speech](#pos)
* [Hidden Markov Models](#hmm)
    * [Maximum likelihood estimation for labelled data](#mle)       
* [Evaluation](#eval)
    * [Marginalisation](#sum)    
    * [Perplexity](#ppl)    
    * [Maximisation](#max)
    * [Accuracy](#acc)

    
**Table of Exercises**

* Model design (-/10)
    * [Exercise 4-1](#ex4-1) (-/2)
    * [Exercise 4-2](#ex4-2) (-/1)
    * [Exercise 4-3](#ex4-3) (-/1)
    * [Exercise 4-4](#ex4-4) (-/1)
    * [Exercise 4-5](#ex4-5) (-/1.5)
    * [Exercise 4-6](#ex4-6) (-/1)
    * [Exercise 4-7](#ex4-7) (-/1.5)
    * [Exercise 4-8](#ex4-8) (-/1)
* MLE (-/20)
    * [Exercise 4-9](#ex4-9) (-/15)
    * [Exercise 4-10](#ex4-10) (-/5)
* Evaluation (-/20)
    * [Exercise 4-11](#ex4-10) (-/1)
    * [Exercise 4-12](#ex4-11) (-/10)
    * [Exercise 4-13](#ex4-12) (-/3)
    * [Exercise 4-14](#ex4-13) (-/1)
    * [Exercise 4-15](#ex4-14) (-/5)



**General notes**

* In this notebook you are expected to use $\LaTeX$. 
* Use python3.
* Use NLTK to read annotated data.
* **Document your code**: TAs are more likely to understand the steps if you document them. If you don't, it's also difficult to give you partial points for exercises that are not completely correct.

After completing this lab you should be able to 

* develop HMM taggers and language models
* estimate parameters via MLE
* marginalise latent variables and evaluate the model intrinsically in terms of perplexity
* predict the best POS tag sequence and evaluate the model in terms of accuracy

# <a name="pos"> Parts of Speech
    
**Parts of speech** (also known as PoS, word classes) give us information about a word and its neighbors. 
PoS can can be divided into: closed class and open class.

Open class words (or **content words**) are nouns, verbs, adjectives, and adverbs, where they refer to objects, actions, and features in the world. They are called open class, since there is no limit to what these words are
new ones are added all the time (email, website, selfie, etc.).

**Nouns** suchs as proper nouns are names of persons of entitnes: *Regina*, *Colorado*,
and *IBM*. Other type are common nouns that refer to objects that for exmaple, can be counted (one car) or homogeneus groups (snow, sand).

**Verbs** consists pf actions and processes, like *draw*, *provide*, and *go*.

***Adjectives** include terms for properties or qualities for concepts like age (*old*, *young*), value (*good*, *bad*).



Closed class words (or **function words**) are pronouns, determiners, prepositions, and connectives. There is a limited number of these.


**Prepositions** occur before noun phrases and indicate spatial or temporal relations, for example, *by* the house.

The PoS are tags that classifiy words. For example in English uses the 36 tags. And these tags are used to manually annotate a wide variety of corpora, the Brown corpus, the Wall Street Journal corpus.

In this Lab we are going to work with the English [Penn Treebank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) tag set and the annotated corpus.

Our final goal is the task: 
* **Part-of-speech tagging** (tagging for short) is the process of assigning a part-ofspeech tag to each word in an input text.

First, we will download the annotated data from [NLTK](https://www.nltk.org/data.html)

In [ ]:
# Read annotated corpora with NLTK
# first download data
import nltk
#nltk.download()
# it will open a GUI and you have to double click in "all" to download 
# this will download different types of annotated corpora

With NLTK, we load the annotated **Penn Treebank** corpus.
This corpus will be used to train and test our PoS taggers. Let's use the last 100 sentences for test.

In [ ]:
# inspect PoS from Treebank
# we use the universal tagset
treebank_sents = nltk.corpus.treebank.tagged_sents(tagset='universal')

# we split our corpus on training, dev and test
treebank_training = list(treebank_sents[:3000]) 
treebank_dev = list(treebank_sents[3000:3100])
treebank_test = list(treebank_sents[3100:])
print(len(treebank_sents))
print(len(treebank_training)) # if it takes too long/memory only use 1000 instances for training!
print(len(treebank_dev)) # we use 100 sentences/instances for validation
print(len(treebank_test)) # we use 814 sentences/instances for test

In [ ]:
# which is the vocabulary?
# we can inspect the vocabulary of the corpus 
vocabulary = set([w for (w, t) in nltk.corpus.treebank.tagged_words(tagset='universal')])
print(len(vocabulary)) # number of words in our corpus

In [ ]:
# we inspect the universal tagset (this is a general mapping becasue each languge use a different tagset.
tagset = set([t for (w, t) in nltk.corpus.treebank.tagged_words(tagset='universal')])
print(tagset) # tags/labes used to annotate the word class

In [ ]:
# The observations are pairs: (sentence, tags)
# so we can use this to get just the sentences (when we need them)
def extract_sentences(treebank_corpus):
    sentences = []
    for observations in treebank_corpus:
        sentences.append([x for x, c in observations])
    return sentences

# The observations are pairs: (sentence, tags)
# so we can use this to get just the tags (when we need them)
def extract_tags(treebank_corpus):
    tags = []
    for observations in treebank_corpus:
        tags.append([c for x, c in observations])
    return tags

<a name="ex4-1" style="color:red">**Exercise 4-1**</a> **[2 points]** 

* **[1 point]** Plot the distribution of POS tags in the PTB and in the Brown corpus. 
* **[1 point]** Plot the distribution of sequence length in the PTB and in the Brown corpus.


<a name="ex4-2" style="color:red">**Exercise 4-2**</a> **[1 point]** Load the PTB corpus, find out for each POS tag what is the most likely tag that follows it. 

<a name="ex4-3" style="color:red">**Exercise 4-3**</a> **[1 point]** Find the most frequent verb on the PTB and on the Brown corpus.

# <a name="hmm"> Hidden Markov Models


The Hidden Markov Model **HMM** is a generative model of sequences. It can be used as a language model to assign probability to sentences, but also as a POS tagger.

As a tagger, an HMM might decide that the following sentence: 

*Mr. Vinken is chairman of Elsevier N.V. the Dutch publishing group.*

should be tagged as in this manually annotated example:

In [ ]:
print(treebank_training[1])

In this lab, you will work with the HMM in both modes, namely, language model and tagger.

Let's start by defining the **HMM**
    
We consider two phenomena:
* Transition: we move from one "state" to another "state" where our state is the POS tag
* Emission: with a certain "state" in mind, we generate a certain word

This means that in the sentence above, for example, we generate
1. From the state `BoS` (begin of sequence) we generate the state `NOUN`
2. Then from `NOUN` we generate the word `Mr.`
3. We then "forget" the word we just emitted and use the fact that our current state is `NOUN` to generate the next state, which is again a `NOUN`
4. From where we then generate `Vinken`
5. We proceed like that until we exaust both sequences


Let us give names to things, let's model the current class with a random variable $C$ and let's use the random variable $C_{\text{prev}}$ to model the previous category. For the word we will use the random variable $X$.
Both $C$ and $C_{\text{prev}}$ take on values in the enumeration of a tagset containing $t$ tags, that is, $\{1, \ldots, t\}$. $X$ takes on values in the enumeration of a vocabulary containing $v$ words, that is, $\{1, \ldots, v\}$.

The **transition** distribution captures how our beliefs in a class vary as a function of the previous class. We will use Categorical distributions for that. In fact, for each possible previous class we get a Categorical distribution over the complete set of classes.

\begin{align}
(1) \qquad C \mid C_{\text{prev}}=p \sim \text{Cat}(\lambda_1^{(p)}, \ldots, \lambda_t^{(p)})
\end{align}
    

<a name="ex4-4" style="color:red">**Exercise 4-4**</a> **[1 point]** What is the probability value of  $P_{C|C_{\text{prev}}}(c|p)$?



<a name="ex4-5" style="color:red">**Exercise 4-5**</a> **[1.5 points]**

* **[0.5 point]** How many cpds do we need in order to represent all transition distributions?
* **[1 point]** What's the representation cost of such a set of distributions? Use [big-O notation](https://en.wikipedia.org/wiki/Big_O_notation).



The **emission** distribution captures how our beliefs in a word vary as a function of the word's class. We will again use Categorical distributions for that. In fact, for each possible class, we get a Categorical distribution over the complete vocabulary.


\begin{align}
(2) \qquad X \mid C=c \sim \text{Cat}(\theta_1^{(c)}, \ldots, \theta_v^{(c)})
\end{align}
   

<a name="ex4-6" style="color:red">**Exercise 4-6**</a> **[1 point]** What's the probability value of $P_{X|C}(x|c)$?


<a name="ex4-7" style="color:red">**Exercise 4-7**</a> **[1.5 points]** 

* **[0.5 point]** How many cpds do we need in order to represent all emission distributions? 
* **[1 point]** What's the representation cost of such a set of distributions? Use [big-O notation](https://en.wikipedia.org/wiki/Big_O_notation).



Now let's turn to the joint distribution $P_{CX|C_{\text{prev}}}$ over classes and words and let's focus on a single step where we can assume the previous class is already available. Then the model factorises as follows:

\begin{align}
(3) \qquad P_{CX|C_{\text{prev}}}(x, c | c_{\text{prev}}) &= P_{C|C_{\text{prev}}}(c|c_{\text{prev}}) P_{X|C}(x|c) 
\end{align}

Therefore, for a tagged sentence $(x_1^n, c_1^n)$ the model assigns the **joint probability**


\begin{align}
(4) \qquad P_{X_1^nC_1^n|N}(x_1^n, c_1^n|n) &= P_{C|C_{\text{prev}}}(c_1|c_0)P_{X|C}(x_1|c_1)P_{C|C_{\text{prev}}}(c_2|c_1)P_{X|C}(x_2|c_1)\cdots P_{C|C_{\text{prev}}}(c_n|c_{n-1})P_{X|C}(x_n|c_n) \\
    &= \prod_{i=1}^n P_{C|C_{\text{prev}}}(c_i|c_{i-1})P_{X|C}(x_i|c_i)
\end{align}

# <a name="mle"> Maximum likelihood estimation for labelled data
    
The HMM is specified by a collection of categorical distributions, some are interpreted as transition distributions, others are intepreted as emission distributions. 
    
As we know by now, MLE for Categorical CPDs is just a matter of counting and dividing.

The MLE solution for the transition distribution is therefore

\begin{align}
(5) \qquad \lambda_c^{(p)} = \frac{\text{count}_{C_{\text{prev}}C}(p, c)}{\sum_{c'=1}^t \text{count}_{C_{\text{prev}C}}(p,c')}
\end{align}

<a name="ex4-8" style="color:red">**Exercise 4-8**</a> **[1 points]** What is the MLE solution for the emission distribution? Note: use the same style as in Equation (5), that is, state the parameter and the solution. 


Now you will **implement** the HMM, this means you will implement *transition distributions*, *emission distributions*, *Laplace smoothing*, *joint probability*, *marginal probability*, and an algorithm for making *predictions*.

# <a name="imp"> Implementation

Here you will implement a language model based on an HMM POS tagger. 

<a name="ex4-9" style="color:red">**Exercise 4-9**</a> **[15 points]** 

You will need to complete the skeleton class below. Read it through before coding. Check the documentation for additional information. Document your steps (this will increase your chance of earning points in case you make mistakes along the way).

* **[8 points]** Implement `estimate_model`: this should take an annotated corpus (such as PTB or Brown) and estimate the CPDs in the model using Laplace smoothing. 
* **[2 points]** Implement `transition_parameter`: see method's documentation
* **[2 points]** Implement `emission_parameter`: see method's documentation
* **[1 points]** Implement `joint_parameter`: see method's documentation
* **[2 points]** Implement `log_joint`: see method's documentation

Show that you methods work by training a model and testing a few examples. This is an example 

```python
treebank_hmm = HMMLM()
treebank_hmm.estimate_model(treebank_training)
print(treebank_hmm.joint_parameter('DET', 'NOUN', 'book'))
sentence = [x for x, _ in treebank_dev[0]]
tag_sequence = [c for _, c in treebank_dev[0]]
print(' '.join(sentence))
print(' '.join(tag_sequence))
print(treebank_hmm.log_joint(sentence, tag_sequence))
```

for which we get

```
9.959527643028553e-05
At Tokyo , the Nikkei index of 225 selected issues , which *T*-1 gained 132 points Tuesday , added 14.99 points to 35564.43 .
ADP NOUN . DET NOUN NOUN ADP NUM VERB NOUN . DET X VERB NUM NOUN NOUN . VERB NUM NOUN PRT NUM .
-193.71018537
```

**Remark** the numbers are an indication, they may vary due to implementation choices (you will not be graded on exactly reproducing numbers).

In [ ]:
import numpy as np
from collections import defaultdict

class HMMLM:
    """
    This is our HMM language model class.
    
    It will be responsible for estimating parameters by MLE
    as well as computing probabilities using the HMM.
    
    We will use Laplace smoothing by default (because we do not want to assign 0 probabilities).
    
    GUIDELINES:
        - by convention we will use the string '-UNK-' for an unknown POS tag
            - and '<unk>' for an unknown word
        - don't forget that with Laplace smoothing the unknown symbols have to be in the support of distributions
        - now you will have 2 types of distributions, so you should deal with unknown symbols for both of them
        - we also need padding for sentences and tag sequences, by convention we will use 
            - '-BOS-' and '-EOS-' for padding tag sequences
            - '<s>' and '</s>' for padding sentences
        - do recall that '-BOS-' is **not** a valid tag
            in other words we never *generate* '-BOS-' tags, we only pretend they occur at
            the 0th position of the tag sequence in order to provide conditioning context
            for the first actual tag
        - similarly, '<s>' is not a valid word
            in other words, we never *generate* '<s>' as a word
            in fact '<s>' is optional as no emission event is based on it
        - on the other hand, '-EOS-' is a valid tag
            you should model it as the last event of a tag sequence
        - similarly, '</s>' is a valid word
            you should consider it as the last event of a sentence
            Note: the last emission is always '-EOS-'' to '</s>', if you are suspecting this is unnecessary 
             because this emission will have probability 1.0, you are right about it! We still suggest modelling 
             it anyway, just to simplify your program (for example, if you model this dummy emission anyway, 
             there will be no need for special treatment of the last position).
            
    You can use whatever data structures you like for cpds
        - we suggest python dict or collections.defaultdict
            but you are free to experiment with list and/or np.array if you like
    """
    
    def __init__(self, transition_alpha=1.0, emission_alpha=1.0):
        self._vocab = set()
        self._tagset = set()
        self._emission_cpds = dict()
        self._transition_cpds = dict()
        self._transition_alpha = transition_alpha
        self._emission_alpha = emission_alpha
        
    def tagset(self):
        """
        Return the tagset: a set of all tags seen by the model (including '-UNK-').
        
        You can modify this if you judge necessary (for example, because you decided  to 
            use different datastructures, but do note that we provide you an implementation
            of the Viterbi algorithm that expects this functionality).        
        """        
        # the -BOS- tag is just something for internal representation
        #  in case you have added it to the tagset, we are removing it here
        #  as keeping it would be bad for algorithms such as Viterbi
        # the -UNK- tag must be in the support (due to Laplace smoothing)
        #  thus in case you forgot it, we are adding it now
        return self._tagset - {'-BOS-'} | {'-UNK-'}
    
    def vocab(self):
        """
        Return the vocabulary of words: all words seen by the model (including '<unk>').
        
        You can modify this if you judge necessary (for example, because you decided  to 
            use different datastructures, but do note that we provide you an implementation
            of the Viterbi algorithm that expects this functionality).        
        """        
        # the <s> token is just something for internal representation
        #  in case you have added it to the vocabulary, we are removing it here
        # the <unk> word must be in the support (due to Laplace smoothing)
        #  thus in case you forgot it, we are adding it now
        return self._vocab - {'<s>'} | {'<unk>'}
        
    def preprocess_sentence(self, sentence, bos=True, eos=True):
        """
        Preprocess a sentence by lowercasing its words and possibly padding it.
        
        :param sentence: a list of tokens (each a string)
        :param bos: if True you will get <s> at the beginning 
        :param eos: if True you will get </s> at the end
        :returns: a list of tokens (lowercased strings)
        """
        # lowercase
        sentence = [x.lower() for x in sentence]
        # optional padding
        if bos: 
            sentence = ['<s>'] + sentence
        if eos:
            sentence = sentence + ['</s>']
        return sentence
        
    def preprocess_tag_sequence(self, tag_sequence, bos=True, eos=True):
        """
        Preprocess a tag sequence with optional padding.
        
        :param tag_sequence: a list of tags (each a string)
        :param bos: if True you will get -BOS- at the beginning 
        :param eos: if True you will get -EOS- at the end
        :returns: a list of tokens 
        """
        # optional padding
        if bos:
            tag_sequence = ['-BOS-'] + tag_sequence
        if eos:
            tag_sequence = tag_sequence + ['-EOS-']
        return tag_sequence
        
    def estimate_model(self, treebank):
        """        
        :param treebank: a sequence of observations as provided by nltk
            each observation is a list of pairs (x_i, c_i)    
            and they have not yet been pre-processed 
        
        Estimate the model parameters.
        
        This method does not have to return anything, it simply computes the necessary cpds.        
        """
        # ***TYPE YOUR SOLUTION HERE***
        pass
        
    
    def transition_parameter(self, previous_tag, current_tag):
        """        
        This method returns the transition probability for tag given the previous tag.
        
        Tips: do not forget that we have a smoothed model, thus 
            - if either tag was never seen, you should pretend it to be '-UNK-'
        
        :param previous_tag: the previous tag (str)
        :param current_tag: the current tag (str)
        :return: transition parameter
        """
        # ***TYPE YOUR SOLUTION HERE***
        pass  
    
    def emission_parameter(self, tag, word):
        """
        This method returns the emission probability for a word given a tag.
        Tips: do not forget that we have a smoothed model, thus 
            - if the tag was never seen, you should pretend it to be '-UNK-'
            - similarly, if the word was never seen, you shoud pretend it to be '<unk>'
        
        :param tag: the current tag (str)
        :param word: the current word (str)
        :return: the emission probability
        """
        # ***TYPE YOUR SOLUTION HERE***
        pass
    
    def joint_parameter(self, previous_tag, current_tag, word):
        """
        This method returns the joint probability of (current tag, word) given the previous tag
            according to Equation (3)
            
        :param previous_tag: the previous tag (str)
        :param current_tag: the current tag (str)
        :param word: the current word (str)
        :returns: P(word, current_tag|previous_tag)
        """
        # ***TYPE YOUR SOLUTION HERE***
        pass
    
    def log_joint(self, sentence, tag_sequence):
        """
        Implement the logarithm of the joint probability over a sentence and tag sequence as in Equation (4)
        
        :param sentence: a sequence of words (each a string) not yet preprocessed
        :param tag_sequence: a sequence of tags (eac a string) not yet preprocessed
        :returns: log P(x_1^n, c_1^n|n)
        """ 
        # ***TYPE YOUR SOLUTION HERE***
        pass 


In [ ]:
treebank_hmm = HMMLM()
treebank_hmm.estimate_model(treebank_training)

In [ ]:
treebank_hmm.log_joint(['i', 'wish'], ['NOUN', '.'])

Example of basic query

In [ ]:
treebank_hmm.joint_parameter('DET', 'NOUN', 'book')

For the example sentence 

In [ ]:
sentence = [x for x, _ in treebank_dev[0]]
tag_sequence = [c for _, c in treebank_dev[0]]
pass

For the whole dev set

In [ ]:
pass

It's useful to criticise the fit of our model to the training data. We can do so by inspecting whether our model managed to capture certain statistics of the data distribution. For example, we can **generate** data using our model and compare the generations to the training data.

To generate data from our model we use a technique known as *ancestral sampling*, this is a sampling algorithm that simply follows the generative story of our model. 

That is, we would sample length first, then for each position we would sample a tag, conditioned on the previous tag, and a word conditioned on the sampled tag, until we are done. Because we do not have an explicit length distribution, an alternative is to sample tags and words until the `EOS` tag is sampled. To prevent infinite loop, it's convenient to set a maximum length, after which the process is interrupted regardless.

Schematically, this looks like

* Starting from $\langle (\text{<s>}, \text{-BOS-}) \rangle$, repeat for at most `max` steps
    * sample $c$ from the distribution $P_{C|C_{\text{prev}}=c_{\text{prev}}}$
    * stop if $c$ is $\text{-EOS-}$
    * otherwise, sample $x$ from the distribution $P_{X|C=c}$
    * concatenate $(x,c)$ to the generation
* Return the generated sample

You will find an implementation of this procedure in the function `ancestral_sample` below.

In [ ]:
import numpy as np

def ancestral_sample(hmm, max_n=100, word_placeholders=True):
    """
    This implements ancestral sampling with a cap on sequence length.
    It returns an annotated sequence, that is, a list of pairs, each pair is a word and its tag.    
    
    If word_placeholders=True, we will use placeholders for words rather than actual samples.
    If you are only interested in inspecting length and POS tags, this is enough and will lead to faster
    runtime. If you are interested in inspecting the actual sentence, set word_placeholders=False.
    """
    
    # just to make sure I remove BOS and fix an arbitrary order using list()
    tagset = list(hmm.tagset() - {'-BOS-'})
    # just to make sure I remove <s> and fix an arbitrary order using list()
    vocab = list(hmm.vocab() - {'<s>'})
    
    # We start from <s>/-BOS-
    sentence = ['<s>']
    tagging = ['-BOS-']
    
    # And these are the sizes of our sample space
    t = len(tagset)    
    v = len(vocab)
    
    # I will be storing cpds as I compute them using numpy objects
    transitions = dict()
    emissions = dict()
    
    # For no more than max_n steps
    for i in range(max_n):             
        # this is the previous tag (str)
        p = tagging[-1]
        # I check if I already have the cpd P(C|C_prev=p)
        cpd_c = transitions.get(p, None)
        if cpd_c is None:  # if not, I create it by filling in a numpy array with transition parameters
            cpd_c = np.array([hmm.transition_parameter(p, c) for c in tagset])
            cpd_c = cpd_c / cpd_c.sum()
            transitions[p] = cpd_c            
        # use numpy to sample following the distribution
        # thus obtaining the current tag (str)
        c = tagset[np.random.choice(t, p=cpd_c)]
        # which is now part of the generated sequence
        tagging.append(c)
        
        # I stop if I sample -EOS-
        if tagging[-1] == '-EOS-':
            sentence.append('</s>') 
            break
                 
        if word_placeholders:  # we might be interested in tag sequences alone
            sentence.append('x[%d]' % (i + 1))
        else:
            # Otherwise I check if I already the cpd P(X|C=c)
            cpd_x = emissions.get(c, None)
            if cpd_x is None:  # if not, I create it by filling in a numpy array with emission parameters
                cpd_x = np.array([hmm.emission_parameter(c, x) for x in vocab])
                cpd_x = cpd_x / cpd_x.sum()
                emissions[c] = cpd_x     
            # use numpy to sample following the distribution
            # thus obtaining the current word (str)
            x = vocab[np.random.choice(v, p=cpd_x)]
            # which is now part of the generated sequence
            sentence.append(x)
        
    # return a list of pairs of kind (word,tag)
    return list(zip(sentence[1:-1], tagging[1:-1]))

Examples of short generations

In [ ]:
ancestral_sample(treebank_hmm, max_n=10, word_placeholders=False)

In [ ]:
ancestral_sample(treebank_hmm, max_n=10, word_placeholders=True)

<a name="ex4-10" style="color:red">**Exercise 4-10**</a> **[5 points]** Generate data from your trained model (aim for something like $1000$ sequences, if your implementation is too slow, $100$ sequences is okay too). For this exercise you can focus on the PTB training corpus. 

* Plot the distribution of POS tags and sentence length in the generated data and compare to the plots obtained directly from the training data. Ideally, in terms of POS tags you generated data should resemble the data distribution pretty well. As for length, the fit is not very good. Can you guess why?
* Inspect if the most sampled verbs are similar to the most frequent verbs in the data.

Note that if you oversmooth your model, there's a higher chance that your plots will differ.

# <a name="eval"> Evaluation
    
We can evaluate the HMM as a language model, where we assess the perplexity of held-out data, or as a POS tagger, where we compute accuracy of predictions against some manually labelled dataset.


## <a name="sum"> Marginalisation
    
In case we use an HMM as a language model, our goal is to assing a probability to a sentence $x_1^n$, regardless of its tag sequence. 
For that we need to marginalise away all possible assignments to $C_1^n$, where every $C_i$ may take on any of the $t$ available tags.

\begin{align}
(6) \qquad P_{S|N}(x_1^n|n) &= \sum_{c_1=1}^t \cdots \sum_{c_n=1}^t P_{X_1^nC_1^n|N}(x_1^n, c_1^n|n) \\
&= \sum_{c_1=1}^t \cdots \sum_{c_n=1}^t  \underbrace{\prod_{i=1}^n P(c_i|c_{i-1})P(x_i|c_i)}_{\text{From Eq (4)}}\\
&=\sum_{c_1=1}^t \cdots \sum_{c_n=1}^t P(c_1|c_0)P(x_1|c_1)P(c_2|c_1)P(x_2|c_1)\cdots P(c_n|c_{n-1})P(x_n|c_n) 
\end{align}

This looks pretty bad! If we have to enumerate all possible tag sequences, there would be just too many of them. That is, in the first sum, $c_1$ takes 1 of $t$ values, then for each of those values $c_2$ will take 1 of $t$ values, and so on. This leads to $t^n$ different tag sequences. An exponential number of them!!! We will never manage to enumerate them, compute their joint probabilities and then sum them up. 

Not by chance we designed the HMM with certain conditional independence assumptions which we can exploit to derive a *tractable* dynamic program. 

[Dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) is a kind of *divide and conquer* strategy. We have to identify sub-problems that we can solve efficiently and maintain a memory of partial solutions that we use to build the final one.

The algorithm that realises this marginalisation efficiently for HMMs is known as the **Forward** algorithm. We essentially look for an efficient answer to the question "what is the total probability so far". We will approach this problem with a recursion that computes the marginal probability for prefix sequences of a certain length.
    
The Forward recursion is pretty simple, if a path has length $0$ we will assume it has probability $1$. If a path has length $i > 0$, we reckon that its total probability is based on the total probability assigned to paths of size $i-1$, where we marginalise over the possible 1-step extensions of those paths. 

For a given (prefix) observation $x_{< i}$, the *forward probability* associated with $(X_i=x_i, C_i=c)$ is the quantity $\alpha_i(c)$ computed as follows:

\begin{align}
(7) \qquad \alpha_i(c) &= 
\begin{cases}
1 & \text{if }i = 0 \\
P_{X|C}(x_i|c) \sum_{p=1}^t \alpha_{i-1}(p) \times P_{C|C_{\text{prev}}}(c|p)  & \text{otherwise}
\end{cases}
\end{align}

This quantity corresponds to the probability of generating the prefix $x_{<i}$ while marginalising out the sequence $C_{<i}$ ($i$th tag not included) and then generating the pair $X_i=x_i, C_i=c$.

This recursive formula can be efficiently implemented to yield marginal probabilities (an iterative implementation is also possible). For a more detailed derivation of the formula refer to [this notebook](Forward.ipynb).

The marginal probability of a sentence $x_1^n$ is therefore $P_{S|N}(x_1^n|n) = \alpha_{n+1}(\text{EoS})$, that is, the probability of $x_1^n$ where:
* we marginalise joint assignments to $C_1^n$ 
* and generate the end of sequence symbols: $X_{n+1}=\text{</s>}, C_{n+1}=\text{EoS}$.

<a name="ex4-11" style="color:red">**Exercise 4-11**</a> **[1 point]** What is the algorithmic complexity of computing the marginal probability of a sentence in the HMM model? 


<a name="ex4-12" style="color:red">**Exercise 4-12**</a> **[10 points]** Implement the forward algorithm for computation of marginal probabilities. For numerical stability, compute the logarithm of the marginal probability of a sentence and realise as many computations as possible in log space. Compute marginal probabilities for a few examples in the dev set.



In [ ]:
def log_marginal(sentence, hmm):
    """
    Return the logarithm of the probability the model assigns to a given sentence.
    
    Feel free to use whatever data structure you like and whatever strategy (iterative or recursive) you like.
    
    Tips:
        - if you are given two log probabilities `a = log p` and `b = log q`, you can obtain the logarithm of
        the product of probabilities by computing `log (p*q) = log p + log q = a + b`.
        - if you are given two log probabilities `a = log p` and `b = log q`, you can obtain the logarithm of 
        the sum of probabilities by computing `log (p + q) = log (exp(log(p)) + exp(log(q)) = log (exp(a) + exp(b))`
        this operation is usually denoted logaddexp(a, b). You can implement it yourself in python 
        or use numpy's logaddexp function.                    
    """
    # ***TYPE YOUR SOLUTION HERE***
    pass

**Sanity check**

There's a naive algorithm for marginal probabilities that is always correct, namely, 

* enumerate all possible tag sequences that are as long as the input sentence
* assess the joint probability of each such tag sequence
* sum their joint probabilities

This is straightforward to implement and can hardly go wrong. However, this is terribly inefficient due to exhaustive enumeration. For a **very short** sentence this is doable, and you can use that as a debugging technique.

Here is an example:

In [ ]:
# Enumerate by hand vs log marginal algorithm
log_prob = float('-inf')
# Enumeration
for p in treebank_hmm.tagset():
    for c in treebank_hmm.tagset():
        # log joint probability
        log_prob = np.logaddexp(log_prob, treebank_hmm.log_joint(['i', 'wish'], [p, c]))
print(log_prob)

Which we can compare to

```python

log_marginal(treebank_hmm, ['i', 'wish'])

```

In [ ]:
#log_marginal(['i', 'wish'], treebank_hmm)

Try marginalising a sequence of size 3, such as `i wish .`

In [ ]:
pass

In [ ]:
#log_marginal(['i', 'wish', '.'], treebank_hmm)

Here for the example sentence

In [ ]:
#sentence = [x for x, _ in treebank_dev[0]]
#print(' '.join(sentence))
#print(log_marginal(sentence, treebank_hmm))

## <a name="ppl"> Perplexity

Perplexity of a model on a test set is the inverse probability of the test set, normalized
by the number of words. Perplexity is a notion of average branching factor, thus a model with low perplexity can be thought of as a *less confused*. That is, each time it introduces a word given some history it picks from a reduced subset of the entire vocabulary (in other words, it is more certain of how to continue). 

If a dataset contains $t$ tokens where $t = \sum_{k=1}^m n_k$, then the perplexity of the dataset is

\begin{align}
(8) \qquad \text{PP}(\mathcal T) &= \left( \prod_{k=1}^m P_{S|N}(\langle x_1^{(k)}, \ldots, x_{n_k}^{(k)} \rangle|n_k; \boldsymbol \theta) \right)^{-1/t}\\
    &= \exp\left(- \frac{1}{t} \sum_{k=1}^m \log P_{S|N}(\langle x_1^{(k)}, \ldots, x_{n_k}^{(k)} \rangle|n_k; \boldsymbol \theta) \right)
\end{align}

where we have already discarded the length distribution (since it's held constant across models). And the probability of the sentence requires marginalising tag sequences as discussed previously.

It's again convenient to realise operations in log space.

You can compare models in terms of the perplexity they assign to the same test data. The lower the perplexity, the better the model is.

<a name="ex4-13" style="color:red">**Exercise 4-13**</a> **[3 points]** Implement `perplexity` below. Train models of PTB and Brown and test both of them on their respective test sets as well as on each other's test set. Report all results.


In [ ]:
def perplexity(sentences, hmm):
    """
    TYPE YOUR SOLUTION
    
    For a dataset of sentences (each sentence is a list of words)
        and an instance of the HMMLM class
        return the perplexity as defined in Equation (8)
    """
    pass

By the way, this is how you load the Brown corpus. Let's use the last 1000 sentences for test. You can reduce the size of the training set if your computer cannot handle what we suggest below.

In [ ]:
# load the Brown corpus wiht the universal tag set
brown_sentences = nltk.corpus.brown.tagged_sents(tagset='universal')

# we split our corpus on training, dev and test
brown_training = list(brown_sentences[:56000]) 
brown_dev = list(brown_sentences[56000:56340])
brown_test = list(brown_sentences[56340:])
print(len(brown_sentences), len(brown_training), len(brown_dev), len(brown_test))

## <a name="max"> Maximisation

In case we use the HMM as a tagger, we are given a sentence $x_1^n$ and are asked for the sequence of tags $c_1^n$ to which the model assign highest probability, that is, we are asked to solve:
    
\begin{align}
(9) \quad    \arg\!\max_{c_1^n \in \{1, \ldots, t\}^n} ~ P_{C_1^n|X_1^n}(c_1^n|x_1^n)
\end{align}

Also note that searching for $c_1^n$ that is maximum under $P_{C_1^n|X_1^n}(c_1^n|x_1^n)$ is equivalent to searching for $c_1^n$ that maximises $\log P_{C_1^n|X_1^n}(c_1^n|x_1^n)$. Think about why that is true. 
Simarly, we can search for $c_1^n$ that maximises $\log P_{X_1^nC_1^n}(x_1^n, c_1^n)$. 
    
    

<a name="ex4-14" style="color:red">**Exercise 4-14**</a> **[1 points]** Why is it that searching for $c_1^n$ that maximises  $\log P_{C_1^n|X_1^n}(c_1^n|x_1^n)$ is exactly equivalent to searching for $c_1^n$ that maximises $\log P_{X_1^nC_1^n}(x_1^n, c_1^n)$? Why is the latter more convenient?

The space of tag sequences we must search through is very large. Consider that we can select any of $t$ tags for each position, thus by simple counting, we are left with $t^n$ possible sequences. Searching through an exponential space is intractable in general. 

Not by chance we designed the HMM with certain conditional independence assumptions, and again those assumptions lead to an efficient dynamic program. 

The algorithm that realises this maximisation efficiently for HMMs is known as the **Viterbi** algorithm, where we need to solve the problem "what is the best probability so far". We will approach this problem with a recursion that computes the probability of the best path of a certain length.
We will use the recursive function $\alpha_i(c)$ which returns the probability of the best tag sequence for the prefix sentence $x_{\le i}$ assuming that its last tag is $C_i=c$.

The Viterbi recursion is pretty simple, if a path has length $0$ we will assume it has probability $1$. If a path has length $i > 0$, we reckon that its maximum probability is based on the maximum probability assigned to paths of size $i-1$, where we extend those paths with one step and check which step yields the best probability.

The Viterbi recursion is formalised below:

\begin{align}
(10)\qquad \alpha_i(c) &= 
\begin{cases}
1 & \text{if }i = 0 \\
P_{X|C}(x_i|c) \times \max_{p \in \{1, \ldots, t\}} \alpha_{i-1}(p) \times P_{C|C_{\text{prev}}}(c|p) & \text{otherwise}
\end{cases}
\end{align}

Note that in the second line, we look for paths of size $i-1$ (which my have ended in one of $t$ possible previous tags) and we try to extend them with tag $c$. The probability of each hypothetical extension is the probability of the best path we mean to extend $\alpha_{i-1}(p)$ times the joint parameters associated with the extension. That is, continuing from a candidate previous best tagging ending in $C_{i-1}=p$ with current class $C_i=c$ (which depends on a transition parameter), and generating the current word $X_i=x_i$ from the current class $C_i=c$ ( which depends on an emission parameter).

Note that the recursive function takes 2 inputs, $i$ which ranges from 1 to $n$, and $c$ which ranges from 1 to $t$. Thus we need to make at most $n \times t$ calls to this function. But also note that each time we call it, we have to solve (in the second line) a maximisation over $t$ possible assignments of the previous tag. Therefore, assuming we do not repeat computations we have already performed, the overall complexity of the algorithm is $O(n \times t^2)$. That's quite an improvement from $O(t^n)$, isn't it?

Of course, crucial to the success of the algorithm is that we use [memoization](https://en.wikipedia.org/wiki/Memoization), a technique by which we store partial solutions and never recompute things.

We provide you with a **recursive** implementation of the Viterbi algorithm which uses the interface of the HMMLM class that you designed above. This implementation is necessary in the experiment with PTB and Brown corpus in the exercises below. Read the implementation carefully to learn from it. We have provided extensive documentation.
One important note is that we compute everything in log space, that's to rely on better numerical properties of log probabilities.

The Viterbi recursion gives us a way to compute the *probability* of the best path. To find the actual best path we just need to traverse the table of $\alpha_{i}(c)$ values starting from $\alpha_{n+1}(\text{EoS})$ and working our way backwards towards $\alpha_0(\text{BoW})$ while looking for the decision (tag) that is best at each point.

In [ ]:
def viterbi(sentence, hmm):
    """
    Computes the best possible tag sequence for a given input
    and also returns its log probability.
    
    This implementation uses recursion.
    
    :returns: tag sequence, log probability
    """
    sentence = hmm.preprocess_sentence(sentence)
    # this is the length (but recall that padding added 2 tokens) 
    n = len(sentence) 
    # we make sure we focus on tags other than BOS and EOS 
    tagset = hmm.tagset() - {'-BOS-', '-EOS-'}

    # we will use a dictionary for convenience
    alpha_table = dict() 

    def alpha(i, c):
        if i == 1:  # BOS is the only possible context for C_1
            return np.log(hmm.joint_parameter('-BOS-', c, sentence[i])), '-BOS-'
        if (i, c) in alpha_table:  # we return previously computed values
            return alpha_table[(i, c)]          
        # or compute yet unknown values
        best_score = -float('inf')
        best_p = None 
        for p in tagset:
            score, _ = alpha(i - 1, p)
            score += np.log(hmm.joint_parameter(p, c, sentence[i]))
            if score > best_score:
                best_score = score
                best_p = p
        alpha_table[(i, c)] = (best_score, best_p)  # store our choice
        return best_score, best_p
    
    # EOS is the only possible tag for </s>
    score, choice = alpha(n-1, '-EOS-')
    prediction = [choice]  # this is the best choice before EOS
    k = n - 2  # and we proceed backwards
    while k > 0:
        _, choice = alpha(k, choice)  # update the previous choice
        prediction.append(choice)  # add to the prediction list
        k -= 1        
    return list(reversed(prediction))[1:], score  # reverse the list and discard BOS

For a very short sentence we can design a sanity check:

In [ ]:
# Enumerate and maximise vs Viterbi
best_log_prob = float('-inf')
best_seq = []
# Enumeration
for pp in treebank_hmm.tagset():
    for p in treebank_hmm.tagset():
        for c in treebank_hmm.tagset():
            # log joint probability
            log_prob = treebank_hmm.log_joint(['i', 'wish', '.'], [pp, p, c])
            if log_prob > best_log_prob:
                best_log_prob = log_prob
                best_seq = [pp, p, c]
print(best_seq, best_log_prob)

In [ ]:
viterbi(['i', 'wish', '.'], treebank_hmm)

In [ ]:
# Here is another test
viterbi_path, viterbi_log_prob = viterbi(['i', 'wish', 'i', 'had', 'a', 'book', '.'], treebank_hmm)
print(viterbi_path, viterbi_log_prob)

##  <a name="acc"> Accuracy

We can evaluate the performance of our tagger by comparing its Viterbi predictions to human annotation. 

For this, we will use the gold standard test data (e.g. treebank_test). Evaluation metrics compute a score for a given model (e.g. our HMM tagger) by comparing the predicted labels that the model generated with the test data againts the gold standard annotation.

The **Accuracy** metric computes the percentage of instances in the test data that our tagger labeled correctly.
If we have a dataset of $m$ labelled sequences
\begin{equation}
\left( \langle x_1^{(k)}, \ldots, x_{n_k}^{(k)}\rangle, \langle \star_1^{(k)}, \ldots, \star_{n_k}^{(k)}\rangle \right)_{k=1}^m
\end{equation}
we can produce all Viterbi predictions
\begin{equation}
\left( \langle x_1^{(k)}, \ldots, x_{n_k}^{(k)}\rangle, \langle c_1^{(k)}, \ldots, c_{n_k}^{(k)}\rangle \right)_{k=1}^m
\end{equation}

and compute

\begin{align}
(11)\qquad \text{accuracy} &= \frac{\sum_{k=1}^m \sum_{i=1}^{n_k} [c_i^{(k)} = \star_i^{(k)}]}{\sum_{k=1}^m n_k}\\
&= \frac{\text{number of correct predictions}}{\text{total tokens}} 
\end{align}



<a name="ex4-15" style="color:red">**Exercise 4-15**</a> **[5 points]** 

* **[1 point]** Implement the accuracy function.
* **[0.5 point]** Compute accuracy on PTB's test set using a PTB-trained model
* **[0.5 point]** Compute accuracy on PTB's test set using a Brown-trained model
* **[0.5 point]** Compute accuracy on Brown's test set using a PTB-trained model
* **[0.5 point]** Compute accuracy on Brown's test set using a Brown-trained model
* **[2 points]** Show examples of correctly and incorrectly predicted sequences and discuss what might have gone wrong in the incorrect cases.

For this exercise you can use the Viterbi implementation we provided (or your own -- its up to you).